In [0]:
# Define the mount point and source information
mount_point = "/mnt/salesdata"
source = "wasbs://salesproject@salesprojectjay.blob.core.windows.net"
configs = {
    "fs.azure.account.key.salesprojectjay.blob.core.windows.net": "y7Re+4u1C5iuGPqA4fkEHvwv26KZCCJdRAVVpgPyBP6+kkVtrQ1uQAe7ty16YdBLVba0E6XI1VSX+ASt+UfynQ=="
}

# Function to check if a mount point exists
def mount_exists(mount_point):
    mounts = dbutils.fs.mounts()
    return any(mount.mountPoint == mount_point for mount in mounts)

# Check if the mount point already exists
if not mount_exists(mount_point):
    dbutils.fs.mount(
        source = source,
        mount_point = mount_point,
        extra_configs = configs
    )
    print(f"Mounted {source} to {mount_point}")
else:
    print(f"{mount_point} is already mounted.")



Out[1]: True

In [0]:
# Load data into DataFrame
df = spark.read.csv("/mnt/salesdata/sales.csv", header=True, inferSchema=True)


In [0]:
# Data cleaning example
df = df.dropna()
df = df.dropDuplicates()
df = df.withColumn("transaction_date", df["transaction_date"].cast("date"))


In [0]:
# Aggregate sales data by region
sales_by_region = df.groupBy("region").sum("amount").withColumnRenamed("sum(amount)", "total_sales")


In [0]:
# Write data to Delta Lake
sales_by_region.write.format("delta").mode("overwrite").save("/mnt/salesdata/delta/sales_by_region")


In [0]:
# Example analysis: top regions by sales
top_regions = sales_by_region.orderBy(sales_by_region["total_sales"].desc())


In [0]:
# Visualize total sales by region
display(top_regions)


region,total_sales
North,730.0
South,530.0
West,420.0
East,400.0


In [0]:
# Register Delta table
spark.sql("CREATE TABLE IF NOT EXISTS sales_by_region USING DELTA LOCATION '/mnt/salesdata/delta/sales_by_region'")

# SQL query
spark.sql("SELECT * FROM sales_by_region ORDER BY total_sales DESC")


Out[9]: DataFrame[region: string, total_sales: double]